## Example usage of get_batch_mask?

In [ ]:
# Example usage:
batch_idx = 2  # Get mask for the 3rd batch (0-based index)
batch_mask = get_batch_mask(torch.tensor(problem.batch_edges), batch_idx)

# Apply the mask to filter edges
batch_edges_filtered = problem.batch_edges[batch_mask]

print(problem.batch_edges[:,0].shape)
print(batch_mask.shape)
#print(torch.tensor(batch_edges_filtered))

(36846,)
torch.Size([36846])


In [ ]:
print(data[0].batch_edges.shape)

print(data[0].n_variables)

batch = []
currentBatch = 0




problem = data[0]
n_variables = problem.n_variables

batch_edges = torch.tensor(problem.batch_edges)           # Shape: (num_edges, 2)

batch_literals = torch.cat([
    torch.arange(0, 40),                   # Positive literals
    torch.arange(n_variables, n_variables + 40)  # Negative literals
])

batch_mask = torch.isin(batch_edges[:, 0], batch_literals)

# Apply the mask
batch_edges_filtered = batch_edges[batch_mask]


print(batch_mask)



"""for i in data[0].batch_edges:
    if i[0] in []"""

## First GT Unsat core generation code

In [ ]:
"""# gt only needs to be calced once, not per epoch!
    countClauses = 0
    reals = []
    gt_edges_per_problem = []
    for current_batch_num in range(len(eval_problem.is_sat)):
            # This can be repeated for each sub_problem
            clauses_current_problem = eval_problem.get_clauses_for_problem(current_batch_num)

            #print(f"Length of sub_problem 0: {len(clauses_current_problem)}")
            # Sub problem 1 Contains literals 11-20, ...
            #print(f"Sub_problem 0 clauses: {clauses_current_problem}")
            
            # Next part is only for calculating unsat_core -> Move to creation of data and save?
            solver = Solver(name='m22')

            offset = eval_problem.n_literals + 1

            # Assumptions must be unique, therefore eval_problem.n_literals + 1
            assumptions = [i + offset for i in range(len(clauses_current_problem))]

            # Add the clauses with selector literals
            for i, clause in enumerate(clauses_current_problem):
                solver.add_clause(clause + [-assumptions[i]])  # Each clause gets a unique assumption
                
            # is_sat not needed right now as all problems should be unsat
            is_sat = solver.solve(assumptions=assumptions)
            
            unsat_core = solver.get_core()

            # Core contains clauses in reverse order, so we reverse it back
            reversed_core = torch.tensor(unsat_core[::-1])
            # Subtract offset from clauses in core to get original clauses
            unsat_core_clauses = reversed_core - offset
                
            # Map back to original clauses
            core_clause_literals = [clauses_current_problem[i] for i in range(len(clauses_current_problem)) if assumptions[i] in unsat_core]

            solver.delete()
            
            # Calculate mask for current sub_problem in eval_problem
            eval_batch_mask = utils.get_batch_mask(torch.tensor(eval_problem.batch_edges), batch_idx=current_batch_num, batch_size=opts['min_n'], n_variables=eval_problem.n_variables)
            
            
            gt_mask = []
            # TODO: THIS LOOKS WRONG
            for i, idx in enumerate(unsat_core_clauses):
                literals = core_clause_literals[i]
                
                # TODO: This only works if all sub_problems have the same amount of clauses!! WRONG!
                # clause = len(clauses_current_problem)*current_batch_num + unsat_core_clauses[i]
                clause = countClauses + unsat_core_clauses[i]
                
                # Sum of problemBatch.n_clauses_per_batch[] before current_batch_num? Or count while calculating gt for data?
                #clause = problemBatch.n_clauses_per_batch[current_batch_num] + unsat_core_clauses[i]
                
                for value in literals:
                    value = value -1 if value >= 1 else eval_problem.n_variables - (value + 1)
                    gt_mask.append([value, clause])
            
            countClauses = countClauses + len(clauses_current_problem)
                    
            sub_problem_edges = eval_problem.batch_edges[eval_batch_mask]
            
            # TODO: VALIDATE THIS!!!
            gt = torch.isin(torch.tensor(sub_problem_edges), torch.tensor(gt_mask))
            # We only need the right column of the isin tensor
            gt = gt[:,1].int()
            
            reals.append(gt.flatten().numpy())
            gt_edges_per_problem.append(gt_mask)
            #reals_unflattened.append(gt)
    
    allReals = np.concatenate(reals)  # Flatten the list of arrays"""

## Test code for calculating unsat cores and MUSs

In [ ]:
from pysat.formula import WCNF
from pysat.formula import CNF
from pysat.examples.musx import MUSX
from pysat.examples.optux import OptUx
import utils

"""problemBatch = data[0]

current_batch_num = 0

clauses_problem_0 = problemBatch.get_clauses_for_problem(current_batch_num)

print(f"Length of sub_problem 1: {len(clauses_problem_0)}")
# Sub problem 1 Contains literals 11-20, ...
print(f"Sub_problem 1 clauses: {clauses_problem_0}")

solver = Solver(name='m22')

#for clause in clauses_problem_0:
#    solver.add_clause(clause)

# TODO: Change this according to length of literals
offset = problemBatch.n_literals + 1

# TODO: Assumptions must be unique! + 100 will not work in batch as there exists clause 100...
assumptions = [i + offset for i in range(len(clauses_problem_0))]

# Add the clauses with selector literals
for i, clause in enumerate(clauses_problem_0):
    solver.add_clause(clause + [-assumptions[i]])  # Each clause gets a unique assumption
    


    
is_sat = solver.solve(assumptions=assumptions)


# TODO: THIS UNSAT CORE REQUIRES AN ASSUMPTION!!! THEREFORE NOT SUITED FOR PGEXPLAINER?! -> Bypass with additional assumption per clause?
unsat_core = solver.get_core()

reversed_core = torch.tensor(unsat_core[::-1])

# TODO: looks like unsat_core is in reverse order, flip?
# Last element of unsat core is first clause in core_clauses

stats = solver.accum_stats()
    
# Map back to original clauses
core_clauses = [clauses_problem_0[i] for i in range(len(clauses_problem_0)) if assumptions[i] in unsat_core]
    
    
cnf = CNF()
for i, clause in enumerate(clauses_problem_0):
    print(clause)
    cnf.append(clause)

print("Computed MUS:")
#print(cnf.to_dimacs())
# Compute Minimally Unsatisfiable Subformulas instead of UNSAT cores? No assumptions needed?
musx = MUSX(cnf)
mus = musx.compute()
print("------------------------------------")
print(f"lenght of computed MUS: {len(mus)}")

with OptUx(cnf) as optux:
    for mus in optux.enumerate():
        print('mus {0} has cost {1}'.format(mus, optux.cost))"""



# TODO: USE MUS INSTEAD OF UNSAT CORE
# Computed mus contains the relative clause numbers for the current problem -> Need to be mapped to edges of original clause
eval_problem = data[0]
sub_problem_start_clause = 0
for current_batch_num in range(len(eval_problem.is_sat)):

    clauses_problem_i = eval_problem.get_clauses_for_problem(current_batch_num)

    cnf = CNF()
    for i, clause in enumerate(clauses_problem_i):
        #print(clause)
        cnf.append(clause)

    musx = MUSX(cnf, verbosity=0)
    # mus contains list of relative clause numbers for the current problem e.g. [4, 7, ..., 53], starting at 1
    mus = musx.compute()
    print(f"computed mus: {mus}")

    eval_batch_mask = utils.get_batch_mask(torch.tensor(eval_problem.batch_edges), batch_idx=current_batch_num, batch_size=opts['min_n'], n_variables=eval_problem.n_variables)

    masked_batch_edges = torch.tensor(eval_problem.batch_edges[eval_batch_mask])
    #print(len(torch.unique(torch.tensor(masked_batch_edges[:, 1]))))
    # ADD offset to mus
    mus_tensor = torch.tensor(mus) + sub_problem_start_clause

    print(mus_tensor)
    
    #gt_mask = torch.zeros(len(masked_batch_edges))
    gt_mask = torch.isin(masked_batch_edges[:, 1],mus_tensor)
    
    masked_batch_edges[gt_mask]
    
    print(gt_mask)
    print(masked_batch_edges)
    print(masked_batch_edges[gt_mask])

    sub_problem_start_clause += len(torch.unique(masked_batch_edges[:, 1]))


"""
solver.delete()

print(is_sat)
#print(stats)
print(f"Computed unsat core length: {len(unsat_core)}")
print(f"Computed unsat core: {unsat_core}")
print("------------------------------------")
print(core_clauses)


utils.visualize_cnf_interactive(clauses_problem_0)



# This is shifted, literals are indexed 10-19 instead of 11-20!!!!!
# positive literals match! First clause (71/70 in batch_edges) is [17, 12, -14, 20]. In batched batch_edges: 16-70, 11-70, 19-70 matches 17, 12, 20
# negative ones do not match!!!!!
print(data[0].batch_edges)

mask = utils.get_batch_mask(torch.tensor(data[0].batch_edges), batch_idx=current_batch_num, batch_size=10, n_variables=problemBatch.n_variables)

masked_batch_edges = data[0].batch_edges[mask]
print(masked_batch_edges)

print(problemBatch.n_variables)
print(problemBatch.n_literals)




# TODO: Create gt in shape of mask with 0 if edge pair not in core, 1 if edge pair in core!
# -> First clause in sub_problem = 70: 17,12,-14,20 in indexes: 16,11,1413,19 has edges 16-17, 11-70, 1413-70 and 19-70
# -> First clause in gt/core = 100 (subtract 100 because of assumption -> 0, first clause(70)) with content [17, 12, -14, 20]. Convert to edge indexes and map!

unsat_core_edges = reversed_core - offset
# connect first element in unsat_core_edges to literals of first core_clauses

gt = []
for i, idx in enumerate(unsat_core_edges):
    literals = core_clauses[i]
    
    # TODO: Swap clauses_problem_0  with current problem?
    # This only works if all sub_problems have the same amount of clauses!! WRONG!
    clause = len(clauses_problem_0)*current_batch_num + unsat_core_edges[i]
    
    # Sum of problemBatch.n_clauses_per_batch[] before current_batch_num? Or count while calculating gt for data?
    #clause = problemBatch.n_clauses_per_batch[current_batch_num] + unsat_core_edges[i]
    
    for value in literals:
        value = value -1 if value >= 1 else problemBatch.n_variables - (value + 1)
        gt.append([value, clause])
        
motif_size = len(gt)

print(torch.tensor(gt))

print(problemBatch.n_clauses_per_batch)

empty_gt = torch.zeros(len(masked_batch_edges))

print(empty_gt.shape)

test = torch.isin(torch.tensor(masked_batch_edges), torch.tensor(gt))

# We only need the right column of the isin tensor
clausesTest = test[:,1].int()
print(clausesTest)


print(problemBatch.n_variables)


gt_mask = []
for i, content in enumerate(masked_batch_edges):
    if masked_batch_edges[i] in unsat_core_edes:
        gt_mask.append(1)
    else:
        gt_mask.append(0)
        
test2 = torch.isin(torch.tensor(data[0].batch_edges), torch.tensor(gt))

# We only need the right column of the isin tensor
allClausesTestgt = test2[:,1].int()
        
        
pos = utils.visualize_edge_index_interactive(masked_batch_edges, clausesTest)
pos2 = utils.visualize_edge_index_interactive(masked_batch_edges, clausesTest, "test", pos, motif_size)"""

computed mus: [5, 8, 9, 12, 23, 31, 38, 43, 45, 52]
tensor([ 5,  8,  9, 12, 23, 31, 38, 43, 45, 52])
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False,  True,  True,  True,  True,  True,  True,  True,  True, False,
        False, False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, False,  True,  True,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True,  True,  True,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False

'\nsolver.delete()\n\nprint(is_sat)\n#print(stats)\nprint(f"Computed unsat core length: {len(unsat_core)}")\nprint(f"Computed unsat core: {unsat_core}")\nprint("------------------------------------")\nprint(core_clauses)\n\n\nutils.visualize_cnf_interactive(clauses_problem_0)\n\n\n\n# This is shifted, literals are indexed 10-19 instead of 11-20!!!!!\n# positive literals match! First clause (71/70 in batch_edges) is [17, 12, -14, 20]. In batched batch_edges: 16-70, 11-70, 19-70 matches 17, 12, 20\n# negative ones do not match!!!!!\nprint(data[0].batch_edges)\n\nmask = utils.get_batch_mask(torch.tensor(data[0].batch_edges), batch_idx=current_batch_num, batch_size=10, n_variables=problemBatch.n_variables)\n\nmasked_batch_edges = data[0].batch_edges[mask]\nprint(masked_batch_edges)\n\nprint(problemBatch.n_variables)\nprint(problemBatch.n_literals)\n\n\n\n\n# TODO: Create gt in shape of mask with 0 if edge pair not in core, 1 if edge pair in core!\n# -> First clause in sub_problem = 70: 17,